In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Format_data").getOrCreate()

from minio import Minio
from minio.commonconfig import Tags

minio_url = 'minio:9000'
access_key = "Qm6pkTNzxbz3Km6QHVwc"
secret_key = "wa3LjHTKv3kQOEg8SQjSivwKinh6PKRVZlnYkUbY"

minio_client = Minio(endpoint=minio_url,
                access_key=access_key,
                secret_key=secret_key,
                secure=False)

b_name = 'warehouse'
list_name_file = []

objects = minio_client.list_objects(
                b_name, recursive=True, prefix="bronze/tracks",
            )

for obj in objects:
    o_name = obj.object_name
    f_name = o_name[14:]
    tags = minio_client.get_object_tags(b_name, o_name)

    try:
        if(tags['Status'] == 'New'):
            try:
                minio_client.fget_object(b_name, o_name, f_name)
                list_name_file.append(f_name)

                tags = Tags.new_object_tags()
                tags["Status"] = "Old"
                minio_client.set_object_tags(b_name, o_name, tags)
            finally:
                pass
    except:
        pass

In [11]:
# For simple, just handle 1 file
f_name = list_name_file[0]

df = spark.read.option("header",True).csv(f_name)

from pyspark.sql.functions import col, regexp_replace
# Sử dụng hàm regexp_replace để xử lý dữ liệu trong cột artist_id
df = df.withColumn("artist_ids", regexp_replace(col("artist_ids"), r"[\[\]']|\s", ""))

from pyspark.sql.functions import split, col
from pyspark.sql.types import ArrayType, StringType

# Định nghĩa hàm UDF (User-Defined Function) để chuyển đổi chuỗi thành mảng
split_udf = split(col("artist_ids"), ", ")
# Đổi cột với kiểu dữ liệu là ArrayType
df2 = df.withColumn("artist_ids", split_udf.cast(ArrayType(StringType())))

# Drop Duplicates
df2 = df2.dropDuplicates()
# Drop Null Values
df2 = df2.na.drop()
# Lọc ra các dòng có cột release_date khác "0000" hoặc là giá trị null
df2 = df2.filter((col("release_date") != "0000"))

# Explode multiple values in artist ids
from pyspark.sql.functions import explode
df1 = df2.withColumn("artist_ids",explode(df2.artist_ids))

# Format colume type
df2 = df1.withColumn("disc_number",df1.disc_number.cast('int'))
df1 = df2
df2 = df1.withColumn("popularity",df1.popularity.cast('int'))
df1 = df2
df2 = df1.withColumn("acousticness",df1.acousticness.cast('float'))
df1 = df2
df2 = df1.withColumn("danceability",df1.danceability.cast('float'))
df1 = df2
df2 = df1.withColumn("duration_ms",df1.duration_ms.cast('int'))
df1 = df2
df2 = df1.withColumn("energy",df1.energy.cast('float'))
df1 = df2
df2 = df1.withColumn("instrumentalness",df1.instrumentalness.cast('float'))
df1 = df2
df2 = df1.withColumn("key",df1.key.cast('int'))
df1 = df2
df2 = df1.withColumn("liveness",df1.liveness.cast('float'))
df1 = df2
df2 = df1.withColumn("loudness",df1.loudness.cast('float'))
df1 = df2
df2 = df1.withColumn("mode",df1.mode.cast('binary'))
df1 = df2
df2 = df1.withColumn("speechiness",df1.speechiness.cast('float'))
df1 = df2
df2 = df1.withColumn("tempo",df1.tempo.cast('float'))
df1 = df2
df2 = df1.withColumn("valence",df1.valence.cast('float'))
df1 = df2
df2 = df1.withColumn("release_date",df1.release_date.cast('date'))

df2.writeTo("silver.spotify_tracks").append()

In [ ]:
%%sql
SELECT * FROM silver.spotify_tracks order by popularity DESC